In [1]:
!pip3 install --upgrade git+https://github.com/retentioneering/retentioneering-tools.git@graph-recalc

  Cloning https://github.com/retentioneering/retentioneering-tools.git (to revision graph-recalc) to /tmp/pip-req-build-4scy8x4e
  Running command git clone -q https://github.com/retentioneering/retentioneering-tools.git /tmp/pip-req-build-4scy8x4e
  Running command git checkout -b graph-recalc --track origin/graph-recalc
  Switched to a new branch 'graph-recalc'
  Branch 'graph-recalc' set up to track remote branch 'graph-recalc' from 'origin'.
     |████████████████████████████████| 10.6 MB 3.5 MB/s eta 0:00:01


  Created wheel for retentioneering: filename=retentioneering-2.0.2.22a0-py3-none-any.whl size=822382 sha256=2c3a31c12ced3678a2c477fdb54906adfea7bfffa2799f586d3451de9593f6e5
  Stored in directory: /tmp/pip-ephem-wheel-cache-o3b9buqb/wheels/6b/4a/11/d93ddeeefef98cece5159e3eb8e4aa8fe9578f50b3ee143351
Successfully built retentioneering
  Attempting uninstall: pandas
    Found existing installation: pandas 1.2.2
    Uninstalling pandas-1.2.2:
      Successfully uninstalled pandas-1.2.2
  Attempting uninstall: retentioneering
    Found existing installation: retentioneering 2.0.2
    Uninstalling retentioneering-2.0.2:
      Successfully uninstalled retentioneering-2.0.2


In [1]:
from retentioneering.core.rete_explorer.rete_explorer import ReteExplorer
import retentioneering

data = retentioneering.datasets.load_simple_shop()


In [2]:
config = {
  'user_col': 'user_id',
  'event_col':'event',
  'event_time_col':'timestamp',
  'custom_cols': ['user_id']
}
rete = ReteExplorer(clickstream=data, config=config)

def norm_events(config, clickstream, nodelist, edgelist):
    d = edgelist["edge_weight"] * 10
    return d

def norm_users(config, clickstream, nodelist, edgelist):
    d = edgelist["user_id"] * 100
    return d
    

rete.use_edgelist_norm_func("edge_weight", norm_events)
rete.use_edgelist_norm_func("user_id", norm_users)


nodelist = rete.create_nodelist()
rete.nodelist = nodelist
edgelist = rete.create_edgelist()
nodelist.head()

,event,number_of_events,user_id,active,alias,parent,changed_name
1,catalog,14518,3611,True,False,None,None
6,main,5635,2385,True,False,None,None
5,lost,3098,3098,True,False,None,None
0,cart,2842,1924,True,False,None,None
12,product2,2172,1430,True,False,None,None


In [3]:
edgelist

,event,next_event,edge_weight,user_id
0,cart,cart,10,100
1,cart,catalog,5710,47800
2,cart,delivery_choice,16860,135600
3,cart,lost,3300,33000
4,cart,main,2330,20400
5,catalog,cart,17090,132400
6,catalog,catalog,48570,200400
7,catalog,lost,16050,160500
8,catalog,main,25450,148000
9,catalog,product1,15150,112200


In [3]:
graph = rete.create_graph(nodelist=nodelist, edgelist=edgelist)

weight_template="events: <%- link.weights.number_of_events.weight  %> users: <%- link.weights.user_id.weight  %>"

graph.plot_graph()

In [7]:
graph.nodelist.columns.tolist()

['event',
 'number_of_events',
 'user_id',
 'active',
 'alias',
 'parent',
 'changed_name']

In [8]:
graph._on_nodelist_updated(graph.updates)

In [10]:
range(len(nodelist.columns.tolist()))

range(0, 7)

In [11]:
nodelist = graph.nodelist
for i in range(len(nodelist.columns.tolist())):
    key = nodelist.columns[i]
    print(key)

event
number_of_events
user_id
active
alias
parent
changed_name


In [20]:
        event_col = graph.rete.config["event_col"]
        nodelist = graph.nodelist
        nodes = graph.updates
        for node in nodes:
            indexes = nodelist.index[nodelist[event_col]
                                     == node['name']].tolist()
            index = indexes[0] if len(indexes) > 0 else None

            print(index)
            if index is not None:
                nodelist.at[index, "active"] = node["active"]
                nodelist.at[index, "parent"] = node["parent"]
                if "changed_name" in node:
                    nodelist.at[index, "changed_name"] = node["changed_name"]
                for col, value in node['degree'].items():
                    nodelist.at[index, col] = value["source"]
            else:
                row: MutableSequence[Any] = [None] * len(nodelist.columns)

                for i in range(len(nodelist.columns.tolist())):
                    key = nodelist.columns[i]
                    print(key)
                    if key == event_col:
                        row[i] = node["name"]
                    elif key == "active":
                        row[i] = node["active"]
                    elif key == "alias":
                        row[i] = node["alias"]
                    elif key == "parent":
                        row[i] = node["parent"]
                    elif key == "changed_name":
                        if "changed_name" in node:
                            row[i] = node["changed_name"]
                        else:
                            row[i] = None
                    elif key in node["degree"]:
                        row[i] = node["degree"][key]["source"]

                nodelist.loc[node["index"]] = row

0
4
8
3
1
6
7
12
5
9
10
11
2


In [16]:
graph.updates

[{'index': 0,
  'name': 'cart',
  'x': 495.74404754680404,
  'y': 384.40027687765576,
  'active': True,
  'alias': False,
  'degree': {'number_of_events': {'degree': 9.872709739633558, 'source': 2842},
   'user_id': {'degree': 19.98449183051786, 'source': 1924}},
  'type': 'suit_node',
  'parent': None},
 {'index': 1,
  'name': 'delivery_pickup',
  'x': 317.99228214595445,
  'y': 470.20562049609964,
  'active': True,
  'alias': False,
  'degree': {'number_of_events': {'degree': 5.045598567295771, 'source': 506},
   'user_id': {'degree': 7.89642758238715, 'source': 469}},
  'type': 'suit_node',
  'parent': None},
 {'index': 2,
  'name': 'payment_cash',
  'changed_name': 'payment_cash123123',
  'x': 432.94517772651704,
  'y': 221.03734886761544,
  'active': True,
  'alias': False,
  'degree': {'number_of_events': {'degree': 4.407080865132938, 'source': 197},
   'user_id': {'degree': 5.578510108003323, 'source': 190}},
  'type': 'suit_node',
  'parent': None},
 {'index': 3,
  'name': 'del

In [4]:
graph.nodelist

,event,number_of_events,user_id,active,alias,parent,changed_name
1,catalog,14518,3611,True,False,None,None
6,main,5635,2385,True,False,None,None
5,lost,3098,3098,True,False,None,None
0,cart,2842,1924,True,False,None,None
12,product2,2172,1430,True,False,None,None
2,delivery_choice,1686,1356,True,False,None,None
11,product1,1515,1122,True,False,None,product1123
9,payment_choice,1107,958,True,False,None,None
3,delivery_courier,834,748,True,False,None,None
10,payment_done,706,653,True,False,None,None
